In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join('../../')))
import logging
# Logging setup
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

module_logger = logging.getLogger()
module_logger.addHandler(handler)
module_logger.setLevel(logging.INFO)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.base import LanguageModel, Chain, Prompt, ProbabilisticComponent, SymbolicComponent, Threads, Concept, ConceptRegistry, use_registry
import pandas as pd
import dtale

# tests

In [4]:
# pd.DataFrame(chain.memory)
# dtale.show(pd.DataFrame(lm3.memory))

# parsers
# list as output
# routers
# retriever


In [5]:
# add placeholders and import 

chain1 = Chain([ProbabilisticComponent(LanguageModel("OpenAI1"), Prompt("tell me a joke about {subject}."), 'joke'), 
                ProbabilisticComponent(LanguageModel("Google1"), Prompt("what do you think of this joke: {joke}?"), 'other'), 
                ProbabilisticComponent(LanguageModel("Microsoft1"), Prompt("can you explain why this joke is funny: {joke}?"), 'subject')])

chain2 = Chain([ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {subject}."), 'joke'), 
                ProbabilisticComponent(LanguageModel("Google2"), Prompt("what do you think of this joke: {joke}?"), 'think'), 
                ProbabilisticComponent(LanguageModel("Microsoft2"), Prompt("can you explain why this joke is funny: {think}?"), 'lala')])

threads1 = Threads([ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {lala}."), 'joke'), 
                ProbabilisticComponent(LanguageModel("Google2"), Prompt("what do you think of this joke: {else}?"), 'think'), 
                ProbabilisticComponent(LanguageModel("Microsoft2"), Prompt("can you explain why this joke is funny: {think}?"), 'kaka')])

component = ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {other}."), 'joke')

# TODO this should be a dict of string to the placeholder 
# TODO collect all placeholders in a dict of name to placeholder
# TODO assign a value to the input placeholders

## Component

In [6]:
Concept('name')

{
    "content": null,
    "inputted": false,
    "level": 0,
    "name": "name"
}

In [7]:
initial_concepts = [Concept(name='other', content='computer'), Concept(name='something', content='something')]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    component.run()

In [8]:
component.memory

['Response of OpenAI2: to (tell me a joke about computer.)']

## Chain

In [9]:
# Run the chain
initial_concepts = [Concept(name='something', content='something'), Concept(name='subject', content='computer')]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    out = chain1.run()

/Users/fuad/Documents/projects/lexflow/src/base/executable.py:153: UserWarning:

Overwrite Warning: 
{
    "content": "computer",
    "inputted": true,
    "level": 0,
    "name": "subject"
}

 Is being overwritten by:
 {
    "content": "Response of Microsoft1: to (can you explain why this joke is funny: Response of OpenAI1: to (tell me a joke about computer.)?)",
    "inputted": false,
    "level": 2,
    "name": "subject"
}



In [10]:
concept_registry

{
    "concepts": {
        "joke": {
            "content": "Response of OpenAI1: to (tell me a joke about computer.)",
            "inputted": false,
            "level": 0,
            "name": "joke"
        },
        "other": {
            "content": "Response of Google1: to (what do you think of this joke: Response of OpenAI1: to (tell me a joke about computer.)?)",
            "inputted": false,
            "level": 1,
            "name": "other"
        },
        "something": {
            "content": "something",
            "inputted": true,
            "level": 0,
            "name": "something"
        },
        "subject": {
            "content": "Response of Microsoft1: to (can you explain why this joke is funny: Response of OpenAI1: to (tell me a joke about computer.)?)",
            "inputted": false,
            "level": 2,
            "name": "subject"
        }
    },
    "initial_concepts": [
        {
            "content": "something",
            "inputted": tru

## Chain of chains

In [11]:
chain3 = Chain([chain1, chain2])

In [12]:

initial_concepts = [Concept(name='subject', content='computer'), Concept(name='else', content='else')]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    chain3.run()
concept_registry

{
    "concepts": {
        "else": {
            "content": "else",
            "inputted": true,
            "level": 0,
            "name": "else"
        },
        "joke": {
            "content": "Response of OpenAI2: to (tell me a joke about computer.)",
            "inputted": false,
            "level": 0,
            "name": "joke"
        },
        "lala": {
            "content": "Response of Microsoft2: to (can you explain why this joke is funny: Response of Google2: to (what do you think of this joke: Response of OpenAI2: to (tell me a joke about computer.)?)?)",
            "inputted": false,
            "level": 2,
            "name": "lala"
        },
        "subject": {
            "content": "computer",
            "inputted": true,
            "level": 0,
            "name": "subject"
        },
        "think": {
            "content": "Response of Google2: to (what do you think of this joke: Response of OpenAI2: to (tell me a joke about computer.)?)",
          

## Threads

In [13]:
initial_concepts = [Concept(name='subject', content='computer'), Concept(name='lala', content='something'), 
                        Concept(name='think', content='something'), Concept(name='else', content='something')]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    threads1._run()


/Users/fuad/Documents/projects/lexflow/src/base/executable.py:153: UserWarning:

Overwrite Warning: 
{
    "content": "something",
    "inputted": true,
    "level": 0,
    "name": "think"
}

 Is being overwritten by:
 {
    "content": "Response of Google2: to (what do you think of this joke: something?)",
    "inputted": false,
    "level": 1,
    "name": "think"
}



In [14]:
concept_registry

{
    "concepts": {
        "else": {
            "content": "something",
            "inputted": true,
            "level": 0,
            "name": "else"
        },
        "joke": {
            "content": "Response of OpenAI2: to (tell me a joke about something.)",
            "inputted": false,
            "level": 1,
            "name": "joke"
        },
        "kaka": {
            "content": "Response of Microsoft2: to (can you explain why this joke is funny: something?)",
            "inputted": false,
            "level": 1,
            "name": "kaka"
        },
        "lala": {
            "content": "something",
            "inputted": true,
            "level": 0,
            "name": "lala"
        },
        "subject": {
            "content": "computer",
            "inputted": true,
            "level": 0,
            "name": "subject"
        },
        "think": {
            "content": "Response of Google2: to (what do you think of this joke: something?)",
           

# More complex

In [15]:
# add placeholders and import 
threads1 = Threads([ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {lala}."), 'joke'), 
                ProbabilisticComponent(LanguageModel("Google2"), Prompt("what do you think of this joke: {something}?"), 'think'), 
                ProbabilisticComponent(LanguageModel("Microsoft2"), Prompt("can you explain why this joke is funny: {think}?"), 'kaka')])

chain2 = Chain([ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {subject}."), 'joke'), 
                ProbabilisticComponent(LanguageModel("Google2"), Prompt("what do you think of this joke: {joke}?"), 'think'), 
                ProbabilisticComponent(LanguageModel("Microsoft2"), Prompt("can you explain why this joke is funny: {think}?"), 'lala'),
                threads1
               ])

chain1 = Chain([ProbabilisticComponent(LanguageModel("OpenAI1"), 
                          Prompt("tell me a joke about {subject}."), 'haha'), 
                ProbabilisticComponent(LanguageModel("Google1"), 
                          Prompt("what do you think of this joke: {haha}?"), 'other'), 
                ProbabilisticComponent(LanguageModel("Microsoft1"), 
                          Prompt("can you explain why this joke is funny: {haha}?"), 'subject'),
                chain2
               ])

component = ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {other}."), 'joke')



In [16]:

# TODO this should be a dict of string to the placeholder 
# TODO collect all placeholders in a dict of name to placeholder
# TODO assign a value to the input placeholders

chain3 = Chain([chain1, chain2, threads1])
initial_concepts = [Concept(name='subject', content='computer'), Concept(name='something', content='something')]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    chain1.run()

/Users/fuad/Documents/projects/lexflow/src/base/executable.py:153: UserWarning:

Overwrite Warning: 
{
    "content": "Response of OpenAI2: to (tell me a joke about Response of Microsoft1: to (can you explain why this joke is funny: Response of OpenAI1: to (tell me a joke about computer.)?).)",
    "inputted": false,
    "level": 3,
    "name": "joke"
}

 Is being overwritten by:
 {
    "content": "Response of OpenAI2: to (tell me a joke about Response of Microsoft2: to (can you explain why this joke is funny: Response of Google2: to (what do you think of this joke: Response of OpenAI2: to (tell me a joke about Response of Microsoft1: to (can you explain why this joke is funny: Response of OpenAI1: to (tell me a joke about computer.)?).)?)?).)",
    "inputted": false,
    "level": 7,
    "name": "joke"
}

/Users/fuad/Documents/projects/lexflow/src/base/executable.py:153: UserWarning:

Overwrite Warning: 
{
    "content": "Response of Google2: to (what do you think of this joke: Respons

In [17]:

# TODO this should be a dict of string to the placeholder 
# TODO collect all placeholders in a dict of name to placeholder
# TODO assign a value to the input placeholders

chain3 = Chain([chain1, chain2, threads1])
initial_concepts = [Concept(name='subject', content='computer'), Concept(name='something', content='something')]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    chain1.run()

In [18]:
concept_registry

{
    "concepts": {
        "something": {
            "content": "something",
            "inputted": true,
            "level": 0,
            "name": "something"
        },
        "subject": {
            "content": "computer",
            "inputted": true,
            "level": 0,
            "name": "subject"
        }
    },
    "initial_concepts": [
        {
            "content": "computer",
            "inputted": true,
            "level": 0,
            "name": "subject"
        },
        {
            "content": "something",
            "inputted": true,
            "level": 0,
            "name": "something"
        }
    ]
}

In [19]:
# object names should be unique otherwise they will be overwritten 

# Listify

In [20]:
# add placeholders and import 
threads1 = Threads([ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {lala}."), 'joke'), 
                ProbabilisticComponent(LanguageModel("Google2"), Prompt("what do you think of this joke: {something}?"), 'think'), 
                ProbabilisticComponent(LanguageModel("Microsoft2"), Prompt("can you explain why this joke is funny: {think}?"), 'kaka')])

chain2 = Chain([ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {subject}."), 'joke'), 
                ProbabilisticComponent(LanguageModel("Google2"), Prompt("what do you think of this joke: {joke}?"), 'think'), 
                ProbabilisticComponent(LanguageModel("Microsoft2"), Prompt("can you explain why this joke is funny: {think}?"), 'lala'),
                threads1
               ])

chain1 = Chain([ProbabilisticComponent(LanguageModel("OpenAI1"), 
                          Prompt("tell me a joke about {subject}."), 'haha'), 
                ProbabilisticComponent(LanguageModel("Google1"), 
                          Prompt("what do you think of this joke: {haha}?"), 'other'), 
                ProbabilisticComponent(LanguageModel("Microsoft1"), 
                          Prompt("can you explain why this joke is funny: {haha}?"), 'subject'),
                chain2
               ])

component = ProbabilisticComponent(LanguageModel("OpenAI2"), Prompt("tell me a joke about {other}."), 'joke')

chain3 = Chain([chain1, chain2, threads1])


In [21]:
# concept_registry

In [22]:
initial_concepts = [Concept(name='other', content='computer'), Concept(name='something', content='something')]
concept_registry = ConceptRegistry(initial_concepts)

chain = Chain([SymbolicComponent(lambda x: [x['other'].content + 'processed', 1], ['other'], ['other_1', 'other_2'])])
with use_registry(concept_registry):
    chain.run()

['other']
{'other': {
    "content": "computer",
    "inputted": true,
    "level": 0,
    "name": "other"
}, 'something': {
    "content": "something",
    "inputted": true,
    "level": 0,
    "name": "something"
}}


In [23]:
concept_registry

{
    "concepts": {
        "other": {
            "content": "computer",
            "inputted": true,
            "level": 0,
            "name": "other"
        },
        "other_1": {
            "content": "computerprocessed",
            "inputted": false,
            "level": 0,
            "name": "other_1"
        },
        "other_2": {
            "content": 1,
            "inputted": false,
            "level": 0,
            "name": "other_2"
        },
        "something": {
            "content": "something",
            "inputted": true,
            "level": 0,
            "name": "something"
        }
    },
    "initial_concepts": [
        {
            "content": "computer",
            "inputted": true,
            "level": 0,
            "name": "other"
        },
        {
            "content": "something",
            "inputted": true,
            "level": 0,
            "name": "something"
        }
    ]
}

In [24]:


chain1 = Chain(
    [Threads(ProbabilisticComponent.create_iterable_from_concept('subject', LanguageModel("OpenAI2"), Prompt("tell me a joke about {subject}."), Concept('joke', type='list')))]
)

c = Concept(name='subject', type='list')
c.assign_content('computer\nkeyboard')

initial_concepts = [c]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    chain1.run()

AttributeError: 'Concept' object has no attribute 'type'

In [ ]:
concept_registry

In [ ]:
comp1 =ProbabilisticComponent.create(**{
        'model': LanguageModel("OpenAI1"),
        'prompt': Prompt("tell me a joke about {subject}."),
        'output_concept': Concept('joke')
    })

chain1 = Chain([comp1])


In [21]:
configs = [
    {
        'model': LanguageModel("OpenAI1"),
        'prompt': Prompt("tell me a joke about {subject}."),
        'output_concept': Concept('joke')
    },
    {
        'model': LanguageModel("OpenAI2"),
        'prompt': Prompt("what do you think of this joke: {joke}?"),
        'output_concept': Concept('think')
    }
    # ... add more configs as needed
]
comp1 = ProbabilisticComponent.create_iterable(configs)
chain1 = Chain(comp1)

In [22]:

# TODO this should be a dict of string to the placeholder 
# TODO collect all placeholders in a dict of name to placeholder
# TODO assign a value to the input placeholders


initial_concepts = [Concept(name='subject', string_content='computer'), Concept(name='something', string_content='something')]
concept_registry = ConceptRegistry(initial_concepts)
with use_registry(concept_registry):
    chain1.run()

In [23]:
concept_registry

ConceptRegistry:
  - subject:
    Name: subject
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe7549f3b50>
    String_content: computer
    List_content: None
    List_concepts: None
    Inputted: True
    Level: 0
  - something:
    Name: something
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe7549f3b50>
    String_content: something
    List_content: None
    List_concepts: None
    Inputted: True
    Level: 0
  - joke:
    Name: joke
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe7549f3b50>
    String_content: Response of OpenAI1: to (tell me a joke about computer.)
    List_content: None
    List_concepts: None
    Inputted: False
    Level: 0
  - think:
    Name: think
    Type: identity
    Choice: all
    Listify_func: <function listify_func at 0x7fe7549f3b50>
    String_content: Response of OpenAI2: to (what do you think of this joke: Response of OpenAI1: to (tell me a

# Data Class experiment

In [23]:
from pydantic.dataclasses import dataclass as pydantic_dataclass

In [26]:
@pydantic_dataclass
class Test:
    something = []

In [31]:
t = Test(e='that')

In [34]:
t

Test()